## The Proxy Pattern
In some applications, we want to execute one or more important action before accessing an object: check the user has privileges, or delay instantiation of a computationally expensive object until the use actually needs it. The pattern gets its name from the proxy (also known as surrogate) object used to perform an important action before accessing the actual object. There are four different well-known proxy types.
* A remote proxy, which acts as the local representation of an object that really exists in a different address space (for example, a network server).
* A virtual proxy, which uses lazy initialization to defer the creation of a computationally expensive object until the moment it is actually needed.
* A protection/protective proxy, which controls access to a sensitive object.
* A smart (reference) proxy, which performs extra actions when an object is accessed. Examples of such actions are reference counting and thread-safety checks.

In [1]:
# virtual proxy example
# The LazyProperty class is actually a descriptor

class LazyProperty(object):
    def __init__(self, method):
        print("LazyProperty class __init__ running!")
        self.method = method
        self.method_name = method.__name__
        # print('function overriden: {}'.format(self.fget))
        # print("function's name: {}".format(self.func_name))

    def __get__(self, obj, cls):
        print("__get__ called!")
        print("obj ", obj, "cls ", cls)
        if not obj:
            return None
        value = self.method(obj)
        # print('value {}'.format(value))
        setattr(obj, self.method_name, value)
        return value


In [2]:
class Test:
    def __init__(self):
        self.x = 'foo'
        self.y = 'bar'
        self._resource = None # we want the resource attribute to load lazily

        
    @LazyProperty
    def resource(self):
        print('initializing self._resource which is: {}'.format(self._resource))
        self._resource = tuple(range(5)) # expensive
        return self._resource

Test.__dict__

LazyProperty class __init__ running!


{'__doc__': None,
 '__init__': <function __main__.__init__>,
 '__module__': '__main__',
 'resource': <__main__.LazyProperty at 0x69dac88>}

In [5]:
def main():
    t = Test()
    print(t.x)
    print(t.y)
    # do more work...
    print(t.resource)
    print(t.resource)
    
if __name__ == "__main__":
    main()

foo
bar
__get__ called!
('obj ', <__main__.Test instance at 0x00000000069F4848>, 'cls ', <class __main__.Test at 0x00000000065E7A08>)
initializing self._resource which is: None
(0, 1, 2, 3, 4)
(0, 1, 2, 3, 4)


A few things to note:
* The _resource variable is initialized not by the time the t instance is created, but the first time that we use t.resource.
* The second time t.resource is used, the variable is not initialized again.

## Another example